In [1]:
#Version 0.0 ---comment preprocessing

import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage as nd
import re

Number_of_entry = 5000
#input data and clean up
data = pd.read_csv('aggression_annotated_comments.tsv', sep = '\t', nrows = Number_of_entry)

comment_data = data.comment
comment_data = np.asarray(comment_data)

PUNCTUATION_NO_SPACE = re.compile("[.;:!*=<>`_'?,\"()\[\]]")
PUNCTUATION_SPACE = re.compile("-")
NEWLINE = re.compile("newlinetoken")
#This is more or less the nltk stop list with negations removed
skip = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", 'im',"youre",
        "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 
        'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
        'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 
        "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 
        'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'if', 'or', 'ive',
        'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 
        'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
        'out', 'on', 'off', 'over', 'under', 'further', 'then', 'once', 'here', 'there', 'theres','when', 'where',
        'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',
        'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
        'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y','u','ur']

for i in range(comment_data.size):
    comment_data[i] = PUNCTUATION_NO_SPACE.sub("",comment_data[i].lower())
    comment_data[i] = NEWLINE.sub("",comment_data[i])
    comment_data[i] = PUNCTUATION_SPACE.sub(" ",comment_data[i].lower())
    comment_data[i] = comment_data[i].split()
    comment_data[i] = [word for word in comment_data[i] if word not in skip]
    
print(comment_data[0])

#stop_list = nltk.corpus.stopwords.words("english")

['not', 'creative', 'dictionary', 'definitions', 'terms', 'insurance', 'ensurance', 'properly', 'applied', 'destruction', 'dont', 'understand', 'fine', 'legitimate', 'criticism', 'ill', 'write', 'three', 'man', 'cell', 'bounty', 'hunter', 'easy', 'understand', 'ensured', 'insured', 'different', 'differ', 'assuredthe', 'sentence', 'quote', 'absolutely', 'neutral', 'arent', 'familiar', 'underlying', 'theory', 'strike', 'back', 'eg', 'submarines', 'employed', 'nuclear', 'warfare', 'guiding', 'insurance', 'nor', 'likely', 'three', 'man', 'cell', 'structure', 'kept', 'ira', 'broken', 'british', 'thats', 'fault', 'fine', 'fix', 'explain', 'but', 'nothing', 'personal', 'creative', 'itim', 'tired', 'arguing', 'article', 'multi', 'party', 'turns', 'plenty', 'use', 'mutually', 'mutual', 'apply', 'standard', 'id', 'moving', 'mutual', 'assured', 'destruction', 'talk', 'not', 'appealing', 'reagan', 'voters', 'biases', 'effectiveness', 'dropping', 'lythere', 'double', 'standard', 'edits', 'comes', '

In [2]:
#Removing different ending of the same word
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

for i in range(comment_data.size):
    lemmatizer = WordNetLemmatizer()
    comment_data[i] = ' '.join([lemmatizer.lemmatize(word) for word in comment_data[i]])
    
print(comment_data[6])

[nltk_data] Downloading package wordnet to /Users/Fenggq/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


removed followingall name early polish ruler ficticious therefore index naming oda von haldensleben husband dagome record first time ruler polanen tribe therefore indicated first document later developing land named polandthis quite comment name fictitious deserves least backing


In [13]:
#Input target for binary model(aggressive or neutral)
annotation = pd.read_csv('aggression_annotations.tsv', sep = '\t', nrows =Number_of_entry*20)
rev_id = annotation.rev_id
target_raw = annotation.aggression
target_pro = np.zeros(Number_of_entry)

ctr = 0
ctr_aggre = 0
ctr_neut = 0
for i in range(target_pro.size):
    ctr2 = 0
    aggr = 0
    curr_rev_id = rev_id[ctr]
    while curr_rev_id == rev_id[ctr]:
        aggr += target_raw[ctr]
        ctr += 1
        ctr2 += 1
    #print(ctr2)
    if (aggr/ctr2 > 0.5):
        target_pro[i] = 1
        ctr_aggre += 1
    else:
        target_pro[i] = 0
        ctr_neut += 1
    
print(target_pro[1])
print('Number of Aggressive comments within the sample:', ctr_aggre)
print('Number of Neutral comments within the sample:', ctr_neut)
print('Percentage of Aggressive comments: %', (ctr_aggre / Number_of_entry)*100)

0.0
Number of Aggressive comments within the sample: 435
Number of Neutral comments within the sample: 4565
Percentage of Aggressive comments: % 8.7


In [4]:
#Calculates target scores based on scores of 2 to -2 
#(most friendly to most aggressive)
score_raw = annotation.aggression_score
score_pro = np.zeros(Number_of_entry)

ctr = 0
for i in range(score_pro.size):
    count = 0
    aggr = 0
    curr_rev_id = rev_id[ctr]
    while curr_rev_id == rev_id[ctr]:
        aggr += score_raw[ctr]
        count += 1
        ctr += 1 
    if (count == 0):
        print(ctr)
        print(rev_id[ctr])
        break
    if (aggr/count > 1.5):
        score_pro[i] = 4
    elif (aggr/count > 0.5):
        score_pro[i] = 3
    elif (aggr/count > -0.5):
        score_pro[i] = 2
    elif (aggr/count > -1.5):
        score_pro[i] = 1
    else:
        score_pro[i] = 0
    
print(score_pro[3])
print(rev_id[30])

2.0
89320


In [5]:
#model for binary predicting
#default TF-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC


#print(type(comment_data[0]))

comment_data = np.asarray(comment_data)


def accuracy_test(c=0.01, ngram=3, trials=100):
    total = 0
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_vectorizer.fit(comment_data)
    comment_vectorized = tfidf_vectorizer.transform(comment_data)

    #print(comment_vectorized.shape)
    for i in range(trials):
        data_train, data_test, target_train, target_test = train_test_split(
            comment_vectorized, target_pro, test_size = 0.1)

        model = LinearSVC(C=0.01)
        model.fit(data_train, target_train)
        total += accuracy_score(target_test, model.predict(data_test))
    accuracy = total / trials
    print("Accuracy: ", accuracy)

accuracy_test()


tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
tfidf_vectorizer.fit(comment_data)
comment_vectorized = tfidf_vectorizer.transform(comment_data)

data_train, data_test, target_train, target_test = train_test_split(
    comment_vectorized, target_pro, test_size = 0.1)

model = LinearSVC(C=0.01)
model.fit(data_train, target_train)
accuracy = accuracy_score(target_test, model.predict(data_test))
print(accuracy)

Accuracy:  0.9138800000000006
0.906


In [6]:
#Words most associated with aggression/friendliness

def agg_friendly_words(model):
    feature_to_coef = {
        word: coef for word, coef in zip(
            tfidf_vectorizer.get_feature_names(), model.coef_[0]
        )
    }

    for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
        print ("Aggressive: ", best_negative)

    for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
        print ("Friendly: ", best_positive)

agg_friendly_words(model)

Aggressive:  ('fuck', 0.2703384064216936)
Aggressive:  ('suck', 0.1252222090999667)
Aggressive:  ('fucking', 0.10578986560631969)
Aggressive:  ('as', 0.0933843099372006)
Aggressive:  ('stop', 0.09177717062683861)
Friendly:  ('article', -0.09295760584540119)
Friendly:  ('would', -0.06025796015081492)
Friendly:  ('page', -0.05404713228773678)
Friendly:  ('name', -0.04882829644682674)
Friendly:  ('but', -0.047712261887129556)


In [7]:
#Training and testing with score data
data_s_train, data_s_test, score_train, score_test = train_test_split(
    comment_vectorized, score_pro, test_size = 0.1)

score_model = LinearSVC(C=0.01)
score_model.fit(data_s_train, score_train)
score_accuracy = accuracy_score(score_test, score_model.predict(data_s_test))
print("Linear SVC", score_accuracy)

lr_model = LogisticRegression(C=1)
lr_model.fit(data_s_train, score_train)
lr_accuracy = accuracy_score(score_test, lr_model.predict(data_s_test))
print("Logistic regression", lr_accuracy)


def accuracy_test(c=0.01, trials=100):
    total = 0
    for i in range(trials):
        data_s_train, data_s_test, score_train, score_test = train_test_split(
            comment_vectorized, score_pro, test_size = 0.1)

        score_model = LinearSVC(C=0.01,multi_class='ovr')
        score_model.fit(data_s_train, score_train)
        total += accuracy_score(score_test, score_model.predict(data_s_test))
    accuracy = total / trials
    print("Accuracy: ", accuracy)

accuracy_test()

def lr_test(c=0.01, trials=100):
    total = 0
    for i in range(trials):
        data_s_train, data_s_test, score_train, score_test = train_test_split(
            comment_vectorized, score_pro, test_size = 0.1)

        score_model = LogisticRegression(C=c,multi_class='ovr')
        score_model.fit(data_s_train, score_train)
        total += accuracy_score(score_test, score_model.predict(data_s_test))
    accuracy = total / trials
    print("LR accuracy: ", accuracy)
#lr_test(c=0.05)
lr_test(c=1)

Linear SVC 0.81


/Users/Fenggq/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Fenggq/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic regression 0.81
Accuracy:  0.8035599999999999
LR accuracy:  0.8063999999999995


In [8]:
agg_friendly_words(model)
print("\n")
agg_friendly_words(score_model)
print("\n")
agg_friendly_words(lr_model)

Aggressive:  ('fuck', 0.2703384064216936)
Aggressive:  ('suck', 0.1252222090999667)
Aggressive:  ('fucking', 0.10578986560631969)
Aggressive:  ('as', 0.0933843099372006)
Aggressive:  ('stop', 0.09177717062683861)
Friendly:  ('article', -0.09295760584540119)
Friendly:  ('would', -0.06025796015081492)
Friendly:  ('page', -0.05404713228773678)
Friendly:  ('name', -0.04882829644682674)
Friendly:  ('but', -0.047712261887129556)


Aggressive:  ('fuck', 0.25676015653637224)
Aggressive:  ('suck', 0.11573207620643464)
Aggressive:  ('bitch', 0.09397524669957837)
Aggressive:  ('fucking', 0.09378996192615134)
Aggressive:  ('as', 0.06673912844083282)
Friendly:  ('article', -0.05938966684943842)
Friendly:  ('not', -0.057118366379594346)
Friendly:  ('page', -0.046755624790953264)
Friendly:  ('would', -0.038240147814675086)
Friendly:  ('one', -0.033250072874161256)


Aggressive:  ('fuck', 5.060456077055475)
Aggressive:  ('suck', 2.5407890398436126)
Aggressive:  ('fucking', 2.0506823583974323)
Aggressi

In [9]:
#model for binary predicting
# TF-idf vectorizer with 'char_wb' analyzer
def accuracy_test_wb(c=0.01, ngram=3, trials=100):
    total = 0
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char_wb')
    tfidf_vectorizer.fit(comment_data)
    comment_vectorized = tfidf_vectorizer.transform(comment_data)

    #print(comment_vectorized.shape)
    for i in range(trials):
        data_train, data_test, target_train, target_test = train_test_split(
            comment_vectorized, target_pro, test_size = 0.1)

        model = LinearSVC(C=0.01)
        model.fit(data_train, target_train)
        total += accuracy_score(target_test, model.predict(data_test))
    accuracy = total / trials
    print("Accuracy: ", accuracy)
    
# TF-idf vectorizer with 'char' analyzer
def accuracy_test_char(c=0.01, ngram=3, trials=100):
    total = 0
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char')
    tfidf_vectorizer.fit(comment_data)
    comment_vectorized = tfidf_vectorizer.transform(comment_data)

    #print(comment_vectorized.shape)
    for i in range(trials):
        data_train, data_test, target_train, target_test = train_test_split(
            comment_vectorized, target_pro, test_size = 0.1)

        model = LinearSVC(C=0.01)
        model.fit(data_train, target_train)
        total += accuracy_score(target_test, model.predict(data_test))
    accuracy = total / trials
    print("Accuracy: ", accuracy)

accuracy_test_wb(ngram=5)
accuracy_test_char(ngram=5)

Accuracy:  0.9126000000000005
Accuracy:  0.9130200000000005


In [10]:
#model for score predicting
# TF-idf vectorizer with 'char' analyzer
def score_test_char(c=0.01, ngram=5, trials=100):
    total = 0
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char')
    tfidf_vectorizer.fit(comment_data)
    comment_vectorized = tfidf_vectorizer.transform(comment_data)

    for i in range(trials):
        data_train, data_test, target_train, target_test = train_test_split(
            comment_vectorized, score_pro, test_size = 0.1)

        model = LinearSVC(C=0.01)
        model.fit(data_s_train, score_train)
        total += accuracy_score(score_test, model.predict(data_s_test))
    accuracy = total / trials
    print("Accuracy: ", accuracy)

# TF-idf vectorizer with 'char_wb' analyzer
def score_test_wb(c=0.01, ngram=5, trials=100):
    total = 0
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='char_wb')
    tfidf_vectorizer.fit(comment_data)
    comment_vectorized = tfidf_vectorizer.transform(comment_data)

    for i in range(trials):
        data_train, data_test, target_train, target_test = train_test_split(
            comment_vectorized, score_pro, test_size = 0.1)

        model = LinearSVC(C=0.01)
        model.fit(data_s_train, score_train)
        total += accuracy_score(score_test, model.predict(data_s_test))
    accuracy = total / trials
    print("Accuracy: ", accuracy)

score_test_wb()
score_test_char()

Accuracy:  0.8100000000000012
Accuracy:  0.8100000000000012
